In [2]:
from constants import *
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import re
import pickle
from gensim.models import Word2Vec, Doc2Vec
from word2vec import EpochLogger, EpochSaver

pd.options.display.max_rows = 500

-----

### re-rimplementation of 'pruned_documents.py'

In [5]:
path = 'NETL/additional_support_files'
#file = 'short_label_documents_de_NETL_implementation'
file = 'short_label_documents_en'
filepath = join(path, file)

with open(filepath, 'rb') as fp:
    lst = pickle.load(fp, encoding='utf8')
    netl = pd.Series(lst)
len(lst), lst[4242002-1]

(4242002, 'random_ep_(number_1)')

In [41]:
title_length = 5 # filter < 5
doc_length = 40 # filter > 40

output_filename = 'short_label_documents_de_NEW_implementation'
goodids = pd.read_pickle(join(ETL_PATH, 'dewiki_good_ids.pickle'))
df = pd.read_pickle(join(ETL_PATH, 'dewiki_new.pickle'))
df = df[[TITLE, TEXT]]
df = df.reset_index(drop=False)
df = df[df.hash_nlp.isin(goodids.index)]
get_len = lambda x: len(x.split())
df.loc[:, 'title_len'] = df.title.map(get_len)
df.loc[:, 'text_len'] = df.text.map(get_len)
df = df.query(f'(title_len < {title_length}) and (text_len > {doc_length})')

titles = df.title.str.replace(' ', '_')
lst_new = titles.tolist()
output_file = 'short_label_documents_de_NEW_implementation'
outpath = join(path, output_file)
print("Writing labels to pickle file", outpath)
with open(outpath, 'wb') as fp:
    pickle.dump(lst_new, fp)
    
titles

Writing labels to pickle file short_label_documents_de_NEW_implementation


0                                             Alan_Smithee
1                                                 Actinium
2                                                  Ang_Lee
3                                                Anschluss
4                                            Aussagenlogik
30                                       Anthony_Minghella
31                                  US-amerikanischer_Film
32                               Vorsätze_für_Maßeinheiten
33                           Abkürzungen/Gesetze_und_Recht
36                                                ISO_4217
37                                            Achsensprung
38                                        Alfred_Hitchcock
39                                          Auteur-Theorie
40                                          Aki_Kaurismäki
41                                                   Anime
42                                              Actionfilm
43                                               Al_Paci

-----

### re-rimplementation of 'get_indices.py'

In [ ]:
doc2vec_indices_file_en = join(path, 'doc2vec_indices_en')
word2vec_indices_file_en = join(path, 'word2vec_indices_en')

# creating output indices file
with open(doc2vec_indices_file_en, 'rb') as fp:
    doc_indices_en = pickle.load(fp)
with open(word2vec_indices_file_en, 'rb') as fp:
    word_indices_en = pickle.load(fp)

word_indices_en

In [4]:
d2vpath = 'NETL/training/trained_models/doc2vec_lemma/wiki_lemma.d2v'
w2vpath = 'NETL/training/trained_models/w2v_lemma/w2v_lemma'
print("Doc2Vec loading", d2vpath)
d2v = Doc2Vec.load(d2vpath)
print("Word2Vec loading", w2vpath)
w2v = Word2Vec.load(w2vpath)
print("models loaded")

Doc2Vec loading NETL/training/trained_models/doc2vec_lemma/wiki_lemma.d2v
Word2Vec loading NETL/training/trained_models/w2v_lemma/w2v_lemma
models loaded


In [30]:
d2v.wv.vocab

{'Alan_Smithee': <gensim.models.keyedvectors.Vocab at 0x7f28a4d93438>,
 'stehen': <gensim.models.keyedvectors.Vocab at 0x7f2898f45c18>,
 'als': <gensim.models.keyedvectors.Vocab at 0x7f2898f45c88>,
 'Pseudonym': <gensim.models.keyedvectors.Vocab at 0x7f2898f45cc0>,
 'für': <gensim.models.keyedvectors.Vocab at 0x7f2898f45cf8>,
 'ein': <gensim.models.keyedvectors.Vocab at 0x7f2898f45d68>,
 'fiktiv': <gensim.models.keyedvectors.Vocab at 0x7f2898f45dd8>,
 'Regisseur': <gensim.models.keyedvectors.Vocab at 0x7f2898f45e10>,
 'der': <gensim.models.keyedvectors.Vocab at 0x7f2898f45e80>,
 'Film': <gensim.models.keyedvectors.Vocab at 0x7f2898f45ef0>,
 'verantworten': <gensim.models.keyedvectors.Vocab at 0x7f2898f45f28>,
 'bei': <gensim.models.keyedvectors.Vocab at 0x7f2898f45f98>,
 'eigentlich': <gensim.models.keyedvectors.Vocab at 0x7f2898f45fd0>,
 'seinen': <gensim.models.keyedvectors.Vocab at 0x7f2890e18080>,
 'Name': <gensim.models.keyedvectors.Vocab at 0x7f2890e180f0>,
 'nicht': <gensim.mode

In [31]:
w2v.wv.vocab

{'Alan_Smithee': <gensim.models.keyedvectors.Vocab at 0x7f288e169b38>,
 'stehen': <gensim.models.keyedvectors.Vocab at 0x7f288e169fd0>,
 'als': <gensim.models.keyedvectors.Vocab at 0x7f288e169f28>,
 'Pseudonym': <gensim.models.keyedvectors.Vocab at 0x7f288e169ef0>,
 'für': <gensim.models.keyedvectors.Vocab at 0x7f288e169eb8>,
 'ein': <gensim.models.keyedvectors.Vocab at 0x7f288e169e10>,
 'fiktiv': <gensim.models.keyedvectors.Vocab at 0x7f288e169da0>,
 'Regisseur': <gensim.models.keyedvectors.Vocab at 0x7f288e169d30>,
 'der': <gensim.models.keyedvectors.Vocab at 0x7f288e169c50>,
 'Film': <gensim.models.keyedvectors.Vocab at 0x7f288e169be0>,
 'verantworten': <gensim.models.keyedvectors.Vocab at 0x7f288e169b70>,
 'bei': <gensim.models.keyedvectors.Vocab at 0x7f288e169ac8>,
 'eigentlich': <gensim.models.keyedvectors.Vocab at 0x7f288e169a90>,
 'seinen': <gensim.models.keyedvectors.Vocab at 0x7f288e169a20>,
 'Name': <gensim.models.keyedvectors.Vocab at 0x7f288e1699b0>,
 'nicht': <gensim.mode

In [38]:
in_file = 'short_label_documents_de_NEW_implementation'
inpath = join(path, in_file)
with open(inpath, 'rb') as fp:
    labels_de = pickle.load(fp)

labels_de

['Alan_Smithee',
 'Actinium',
 'Ang_Lee',
 'Anschluss',
 'Aussagenlogik',
 'Anthony_Minghella',
 'US-amerikanischer_Film',
 'Vorsätze_für_Maßeinheiten',
 'Abkürzungen/Gesetze_und_Recht',
 'ISO_4217',
 'Achsensprung',
 'Alfred_Hitchcock',
 'Auteur-Theorie',
 'Aki_Kaurismäki',
 'Anime',
 'Actionfilm',
 'Al_Pacino',
 'Alkohole',
 'The_Visitors',
 'Aluminium',
 'Antimon',
 'Argon',
 'Arsen',
 'Astat',
 'Alkalimetalle',
 'Actinoide',
 'Americium',
 'Atom',
 'Arzt',
 'Anthropologie',
 'Alexander_der_Große',
 'Antike',
 'Anthony_Hope',
 'Ångström',
 'Ampere',
 'Acre',
 'Ar',
 'Arbeit',
 'Atomare_Masseneinheit',
 'Anglizismus',
 'Astronom',
 'Alan_Turing',
 'Archäologie',
 'Außenbandruptur_des_oberen_Sprunggelenkes',
 'Alphabet',
 'Arthur_Harris',
 'Astronomie',
 'Angelina_Jolie',
 'Archimedes',
 'Aristoteles',
 'Abraham_Lincoln',
 'Altweibersommer',
 'Astana',
 'Angela_Merkel',
 'Alicia_Silverstone',
 'Al-Biruni',
 'Ankara',
 'Atheismus',
 'Anna_Seghers',
 'Atmosphäre',
 'Arvo_Pärt',
 'Afghan

In [ ]:
doc_indices_de = []
word_indices_de = []

dout = []
wout = []
for label in labels_de[:]:
    try:
        idx = d2v.wv.vocab[label].index
        doc_indices_de.append(idx)
    except:
        dout.append(label)
    try:
        idx = w2v.wv.vocab[label].index
        word_indices_de.append(idx)
    except:
        wout.append(label)
        # print(f'{label} not in w2v vocab')

doc_indices_de

In [49]:
# most of the labels are OOV, probably due to min_count=20 (that parametrization is from the NETL implementation)
len(doc_indices_de), len(dout), len(word_indices_de), len(wout)

(362555, 1299191, 362555, 1299191)

In [51]:
doc2vec_indices_file_de = join(path, 'doc2vec_indices_de')
word2vec_indices_file_de = join(path, 'word2vec_indices_de')

# creating output indices file
with open(doc2vec_indices_file_de, 'wb') as fp:
    pickle.dump(doc_indices_de, fp)
with open(word2vec_indices_file_de, 'wb') as fp:
    pickle.dump(word_indices_de, fp)

-----
### Pagerank titles -> remove categories from title phrases

In [18]:
path = '../../master_cloud/external_code/NETL-Automatic-Topic-Labelling-/model_run/support_files/de/'
df = pd.read_csv(join(path, 'pagerank-titles-sorted_de.txt'), sep='\t', header=None, keep_default_na=False)

def process_titles(title):
    title = title.split('(')[0].strip('_')
    return title

df[1] = df[1].map(process_titles)
df.to_csv(join(path, 'pagerank-titles-sorted_de_categories_removed.txt'), sep='\t', header=None, index=None)